<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/TFQ_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq tensorflow_quantum

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy as sp

# Qubits and circuit
q0, q1 = cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)
theta = sp.Symbol('theta')

circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.rx(theta)(q0),
    cirq.ry(theta)(q1),
)

readout_ops = [cirq.Z(q0)]

# Keras model: circuits only -> PQC learns theta
inputs = tf.keras.Input(shape=(), dtype=tf.string, name="circuits")
expvals = tfq.layers.PQC(circuit, readout_ops)(inputs)  # shape [batch, 1]
outputs = tf.keras.layers.Dense(2, activation="softmax")(expvals)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Synthetic labels (binary)
n = 1000
x_circ = tfq.convert_to_tensor([cirq.Circuit()] * n)
y = np.random.randint(0, 2, size=(n,)).astype(np.int32)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(x_circ, y, epochs=10, batch_size=32, verbose=1)

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy as sp

# Qubits and symbols
q0, q1 = cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)
th0, th1 = sp.symbols('th0 th1')

# Base circuit + parameterized rotations
circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.CNOT(q0, q1),
    cirq.rx(th0)(q0),
    cirq.ry(th1)(q1),
)
readout_ops = [cirq.Z(q0)]

class ParadoxicalQuantumAI(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # ControlledPQC maps features to the symbols [th0, th1]
        self.pqc = tfq.layers.ControlledPQC(circuit, readout_ops)
        self.head = tf.keras.layers.Dense(2, activation="softmax")

    def call(self, inputs):
        circ_inp, feat_inp = inputs  # feat_inp shape: [batch, 2]
        expvals = self.pqc([circ_inp, feat_inp])
        return self.head(expvals)

# Data: circuits + 2D features + labels
n = 1000
x_circ = tfq.convert_to_tensor([cirq.Circuit()] * n)
x_feat = np.random.uniform(-np.pi, np.pi, size=(n, 2)).astype(np.float32)
y = np.random.randint(0, 2, size=(n,)).astype(np.int32)

model = ParadoxicalQuantumAI()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit([x_circ, x_feat], y, epochs=10, batch_size=32, verbose=1)